In [165]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_columns', 10, 'max_rows', 20)

In [166]:
import zipfile

zf = zipfile.ZipFile('./data/mba_challenge.zip', 'r')
tr_train = pd.read_csv(zf.open('X_train.csv'), encoding='cp949')
tr_test = pd.read_csv(zf.open('X_test.csv'), encoding='cp949')
tr = pd.concat([tr_train, tr_test])
tr

,custid,sales_date,sales_time,str_nm,goodcd,...,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
0,0,2000-06-25 00:00:00,1212,무역점,2116050008000,...,90000,9000,81000,3,0
1,0,2000-06-25 00:00:00,1242,무역점,4125440008000,...,39000,3900,35100,1,0
2,0,2000-08-26 00:00:00,1810,본점,2116052008000,...,175000,17500,157500,3,0
3,0,2000-08-26 00:00:00,1830,본점,4106430119900,...,455000,45500,409500,3,0
4,0,2000-09-03 00:00:00,1802,무역점,2139141008000,...,100000,10000,90000,3,0
5,0,2000-09-03 00:00:00,1842,무역점,4300381019900,...,113000,11300,101700,3,0
6,0,2000-09-03 00:00:00,1930,무역점,6315960019900,...,118000,11800,106200,3,0
7,0,2000-12-13 00:00:00,1920,본점,2116052008000,...,96000,9600,86400,3,0
8,0,2000-12-22 00:00:00,1910,본점,2116052008000,...,166000,16600,149400,3,0
9,0,2000-12-29 00:00:00,1900,본점,2116052008000,...,96000,9600,86400,3,0


In [167]:
tr_train.columns

Index(['custid', 'sales_date', 'sales_time', 'str_nm', 'goodcd', 'brd_nm',
       'corner_nm', 'pc_nm', 'part_nm', 'team_nm', 'buyer_nm', 'import_flg',
       'tot_amt', 'dis_amt', 'net_amt', 'inst_mon', 'inst_fee'],
      dtype='object')

In [168]:
tr[['corner_nm', 'part_nm', 'goodcd']]

,corner_nm,part_nm,goodcd
0,수입종합화장품,명품잡화,2116050008000
1,수입종합화장품,명품잡화,4125440008000
2,수입종합화장품,잡화파트,2116052008000
3,수입의류,잡화파트,4106430119900
4,수입종합화장품,명품잡화,2139141008000
5,캐릭터캐주얼,남성의류,4300381019900
6,트래디셔널,골프/유니캐쥬얼,6315960019900
7,수입종합화장품,잡화파트,2116052008000
8,수입종합화장품,잡화파트,2116052008000
9,수입종합화장품,잡화파트,2116052008000


In [211]:
from sklearn.decomposition import PCA

def dummy_to_pca(tr, column_name:str, features) :
    max_seq = 300
    max_d = 10
    col_count = tr.groupby(column_name)[column_name].count()
    if len(col_count) > max_seq:
        tops = col_count.sort_values(ascending=False)[0:max_seq].index
        f =tr.loc[tr[column_name].isin(tops)][['custid', column_name]]
    else:
        tops = col_count.index
        f =tr[['custid', column_name]]
    f = pd.get_dummies(f, columns=[column_name])  # This method performs One-hot-encoding
    f = f.groupby('custid').mean()
    if len(tops) < max_d:
        max_d = len(tops)
    pca = PCA(n_components=max_d)
    pca.fit(f)
    cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합
    #print(cumsum)
    num_d = np.argmax(cumsum >= 0.99) + 1 # 분산의 설명량이 99%이상 되는 차원의 수
    if num_d == 1:
        num_d = max_d
    pca = PCA(n_components=num_d)    
    result = pca.fit_transform(f)
    result = pd.DataFrame(result)
    result.columns = [column_name + '_' + str(column) for column in result.columns]
    result.index = f.index
    return pd.concat([features, result], axis=1, join_axes=[features.index])
#f = dummy_to_pca(tr, 'team_nm', f)
#f

In [193]:
f = tr.groupby('custid').agg({
    'tot_amt': [('총구매액', 'sum'),('구매건수', 'size'),('평균구매가격', 'mean'),('최대구매액', 'max')],
    'dis_amt': [('dis_sum', 'sum'),('dis_mean', 'mean')],
    'net_amt': [('net_sum', 'sum'),('net_mean', 'mean')],
    'inst_mon': [('평균할부개월수', 'mean'), ('최대할부개월수', 'max')],
    'brd_nm': [('구매상품다양성', lambda x: x.nunique()), 
               ('구매상품다양성비', lambda x: x.nunique()/x.count())],
    'import_flg': [('수입상품_구매비율', "mean"), ('수입상품_구매수', 'sum')],
    'sales_date': [
        ('내점일수',lambda x: x.str[:10].nunique()),
        ('내점비율',lambda x: x.str[:10].nunique()/x.count()),
        ('주말방문비율', lambda x: np.mean(pd.to_datetime(x).dt.dayofweek>4)),
        ('봄-구매비율', lambda x: np.mean( pd.to_datetime(x).dt.month.isin([3,4,5]))),
        ('여름-구매비율', lambda x: np.mean( pd.to_datetime(x).dt.month.isin([6,7,8]))),
        ('가을-구매비율', lambda x: np.mean( pd.to_datetime(x).dt.month.isin([9,10,11]))),
        ('겨울-구매비율', lambda x: np.mean( pd.to_datetime(x).dt.month.isin([1,2,12])))
    ],
    'sales_time': [('밤구입비율', lambda x: np.count_nonzero(x.astype(np.int)[(x>1800)|(x<900)])/ x.count())],
    }).reset_index()
f.columns = f.columns.get_level_values(1)
f.rename(columns={'': 'custid'}, inplace=True)
f = dummy_to_pca(tr, 'brd_nm', f)
f = dummy_to_pca(tr, 'corner_nm', f)
f = dummy_to_pca(tr, 'pc_nm', f)
f = dummy_to_pca(tr, 'part_nm', f)
f = dummy_to_pca(tr, 'buyer_nm', f)
f = dummy_to_pca(tr, 'team_nm', f)
f = dummy_to_pca(tr, 'goodcd', f)
f = dummy_to_pca(tr, 'str_nm', f)
tr['month'] = pd.to_datetime(tr['sales_date']).dt.month.astype(str)
f = dummy_to_pca(tr, 'month', f)
tr['week'] = pd.to_datetime(tr['sales_date']).dt.dayofweek.astype(str)
f = dummy_to_pca(tr, 'week', f)
tr['time'] = np.floor(tr['sales_time']/100).astype(int).astype(str)
f = dummy_to_pca(tr, 'time', f)
f

,custid,총구매액,구매건수,평균구매가격,최대구매액,...,time_4,time_5,time_6,time_7,time_8
0,0,1742000,11,158363.636364,455000,...,-0.027192,-0.051028,-0.042444,-0.116882,-0.011436
1,1,2772100,26,106619.230769,393000,...,-0.121652,-0.084546,-0.074494,0.010964,0.022739
2,2,3750850,11,340986.363636,1416000,...,0.079097,0.056289,-0.019747,0.098974,0.162818
3,3,2300500,30,76683.333333,621000,...,-0.081944,0.016371,-0.056145,0.013360,-0.021576
4,4,1045000,4,261250.000000,560000,...,0.223958,0.648055,-0.415265,-0.059173,-0.007642
5,5,5053759,32,157929.968750,682000,...,-0.003776,-0.075112,-0.057431,0.171091,-0.045622
6,6,3785029,31,122097.709677,936000,...,-0.084492,0.012515,0.053648,-0.067849,-0.006625
7,7,1223182,35,34948.057143,202000,...,0.093355,-0.078708,0.082633,0.008867,-0.005180
8,8,1267500,18,70416.666667,400000,...,0.085882,0.058006,-0.098798,-0.054982,-0.006221
9,9,4956620,59,84010.508475,395000,...,-0.082948,-0.082045,-0.030690,-0.154460,-0.014285


In [195]:
X_train = pd.DataFrame({'custid': tr_train.custid.unique()})
X_train = pd.merge(X_train, f, how='left')
display(X_train)

X_test = pd.DataFrame({'custid': tr_test.custid.unique()})
X_test = pd.merge(X_test, f, how='left')
display(X_test)

,custid,총구매액,구매건수,평균구매가격,최대구매액,...,time_4,time_5,time_6,time_7,time_8
0,0,1742000,11,158363.636364,455000,...,-0.027192,-0.051028,-0.042444,-0.116882,-0.011436
1,1,2772100,26,106619.230769,393000,...,-0.121652,-0.084546,-0.074494,0.010964,0.022739
2,2,3750850,11,340986.363636,1416000,...,0.079097,0.056289,-0.019747,0.098974,0.162818
3,3,2300500,30,76683.333333,621000,...,-0.081944,0.016371,-0.056145,0.013360,-0.021576
4,4,1045000,4,261250.000000,560000,...,0.223958,0.648055,-0.415265,-0.059173,-0.007642
5,5,5053759,32,157929.968750,682000,...,-0.003776,-0.075112,-0.057431,0.171091,-0.045622
6,6,3785029,31,122097.709677,936000,...,-0.084492,0.012515,0.053648,-0.067849,-0.006625
7,7,1223182,35,34948.057143,202000,...,0.093355,-0.078708,0.082633,0.008867,-0.005180
8,8,1267500,18,70416.666667,400000,...,0.085882,0.058006,-0.098798,-0.054982,-0.006221
9,9,4956620,59,84010.508475,395000,...,-0.082948,-0.082045,-0.030690,-0.154460,-0.014285


,custid,총구매액,구매건수,평균구매가격,최대구매액,...,time_4,time_5,time_6,time_7,time_8
0,30000,2078240,27,76971.851852,448000,...,-0.090938,-0.061237,0.005501,0.027377,-0.013290
1,30001,4158320,27,154011.851852,812000,...,0.159615,-0.022233,-0.005896,-0.004410,-0.004460
2,30002,8007256,100,80072.560000,410000,...,0.014761,-0.060047,-0.030018,-0.024924,-0.015133
3,30003,1367820,43,31809.767442,138000,...,0.090662,0.113450,-0.086375,-0.069067,-0.007766
4,30004,2890471,55,52554.018182,1110000,...,0.016713,0.051253,-0.055942,-0.011423,0.004898
5,30005,57000,1,57000.000000,57000,...,0.393321,0.004256,0.001269,-0.000960,-0.018367
6,30006,589750,7,84250.000000,303000,...,-0.263447,0.046095,0.063999,0.125325,-0.026488
7,30007,295500,6,49250.000000,65000,...,-0.089892,-0.040499,0.017714,0.001165,-0.008637
8,30008,400220,9,44468.888889,159200,...,-0.131971,-0.048409,0.198176,0.037374,-0.005465
9,30009,239000,4,59750.000000,157000,...,-0.311018,0.177262,-0.112113,-0.008799,-0.004593


In [196]:
IDtest = X_test.custid;
X_train.drop(['custid'], axis=1, inplace=True)
X_test.drop(['custid'], axis=1, inplace=True)
y_train = pd.read_csv(zf.open('y_train.csv')).gender

In [210]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

#parameters = {'max_depth': 6, 'n_estimators': 200}
#clf = RandomForestClassifier(**parameters, random_state=0)

#parameters = {'xgb__max_depth': 3, 'xgb__subsample': 0.7}
parameters = {'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 1.0, 'learning_rate': 0.05, 
              'min_child_weight': 5, 'silent': True, 'n_estimators': 200}
clf = XGBClassifier(**parameters, random_state=0, n_jobs=-1)
# clf = XGBClassifier()
# parameters = {
#     'max_depth': [4],
#     'subsample': [0.9],
#     'colsample_bytree': [1.0],
#     'learning_rate' : [0.05],
#     'min_child_weight': [5],
#     'silent': [True],
#     'n_estimators': [200]
# }
# clf = GridSearchCV(clf, parameters, n_jobs=1, cv=2)
# clf.fit(X_train, y_train)
# best_est = clf.best_estimator_
# print(best_est)
score = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')

print('{}\nmean = {:.5f}\nstd = {:.5f}'.format(score, score.mean(), score.std()))

[0.69085339 0.69534613 0.69856836 0.70967554 0.68860103]
mean = 0.69661
std = 0.00739


In [ ]:
pred = clf.fit(X_train, y_train).predict_proba(X_test)[:,1]
fname = './data/submission.csv'
submissions = pd.concat([IDtest, pd.Series(pred, name="gender")] ,axis=1)
submissions.to_csv(fname, index=False)
print("'{}' is ready to submit." .format(fname))